In [25]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from sklearn.metrics import confusion_matrix
from keras.layers import Input
from keras.models import Model
from keras.utils import multi_gpu_model
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from imutils import paths
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import tensorflow as tf
#For injection
import keras.backend.tensorflow_backend as tfback

In [26]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [27]:
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

In [28]:
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

In [29]:
trainPath = '../FuelTypeData/train/'
testPath = '../FuelTypeData/test/'
valPath = '../FuelTypeData/vald'
BATCH_SIZE = 128
PFR_NUM_CLASS = 10
FUEL_TYPE_NUM_CLASS = 5
EPOCHS = 150

In [30]:
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(testPath)))
totalTest = len(list(paths.list_images(valPath)))

In [31]:
def plot_training(H, N, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

## Augmentation

In [32]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator('''
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"''')
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

## Generator

In [33]:
# initialize the training generator
img_size = 224
trainGen = trainAug.flow_from_directory(
    trainPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)
# initialize the validation generator
valGen = valAug.flow_from_directory(
    valPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)
# initialize the testing generator
testGen = valAug.flow_from_directory(
    testPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)

Found 8557 images belonging to 5 classes.
Found 1063 images belonging to 5 classes.
Found 1063 images belonging to 5 classes.


In [34]:
# load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(img_size, img_size, 3)))
# construct the head of the model that will be placed on top of the the base model


In [35]:
#baseModel =  pickle.load(open('../Models/VGG16.sav', 'rb'))

In [36]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [37]:
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(FUEL_TYPE_NUM_CLASS, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# compile our model (this needs to be done after our setting our layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
#initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=EPOCHS)
# reset the testing generator and evaluate the network after
# fine-tuning just the network head

[INFO] compiling model...
[INFO] training head...
Epoch 1/150
66/66 [==============================] - 97s 1s/step - loss: 10.9594 - accuracy: 0.5160 - val_loss: 0.1421 - val_accuracy: 0.8496
Epoch 2/150
66/66 [==============================] - 92s 1s/step - loss: 0.8612 - accuracy: 0.8014 - val_loss: 0.8349 - val_accuracy: 0.8353
Epoch 3/150
32/66 [=============>................] - ETA: 46s - loss: 0.3864 - accuracy: 0.8665

In [ ]:
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
    steps=(totalTest // BATCH_SIZE) + 1)
predIdxsClasses = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxsClasses,
    target_names=testGen.class_indices.keys()))

In [ ]:
labs = testGen.class_indices.keys()
FTLabels = []
for lab in labs:
    FTLabels.append(lab)
FTLabels

In [ ]:
cm = confusion_matrix(testGen.classes, predIdxsClasses)
cm = pd.DataFrame(cm)
cm.columns = FTLabels
cm.index = FTLabels
cm

In [ ]:
WARMUP_PLOT_PATH = '../Models/FuelTypeModel/train.png'
plot_training(H, EPOCHS, WARMUP_PLOT_PATH)

In [ ]:
pickle.dump(model, open('../Models/FuelTypeModel/FTModel.sav', 'wb'))